# Домашнее задание 4

## Глубинное обучение в анализе графовых данных, ПМИ ВШЭ

In [1]:
# import torch

# !pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
# !pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
# !pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
# !pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
# !pip install git+https://github.com/pyg-team/pytorch_geometric.git

## 1. Реализация TransE (10 баллов)

В этом задании требуется реализовать пайплайн обучения эмбдеддингов графа знаний с помощью [TransE](https://proceedings.neurips.cc/paper/2013/file/1cecc7a77928ca8133fa24680a88d2f9-Paper.pdf) для задачи прогнозирования отсутствующих ребер на наборе данных [Freebase](https://paperswithcode.com/dataset/fb15k) (FB15k-237), а также реализовать саму модель.

In [21]:
import torch_geometric
from torch_geometric.datasets.rel_link_pred_dataset import RelLinkPredDataset

dataset = RelLinkPredDataset('data', 'FB15k-237')
data = dataset[0]

#### TransE
Ребра в графе знаний представляются тройками $(h, r, t)$. В TransE мы моделируем как объекты, так и отношения в пространстве эмбеддингов и пытаемся получить эмбеддинги, как $\textbf h + \textbf l \approx \textbf t$. Формально loss выглядит:

$$\sum_{((h, l, t), (h', l, t')) \in T_{batch}} [\gamma + d(\textbf{h} + \textbf{l}, \textbf t) - d(\textbf{h'} + \textbf l, \textbf{t'})]$$

где $(h', l, t')$ представляет собой тройку, заменяя head или tail случайным объектом.
$d(\boldsymbol{h}+\boldsymbol{l}, \boldsymbol{t})$ – показатель _различия_ положительного ребра. Кроме того, $d\left(\boldsymbol{h}^{\prime}+\boldsymbol{l}, \boldsymbol{t}^{\prime}\right)$ — это оценка _различия_ для отрицательной тройки, полученная изменением либо head, либо tail (но не оба) положительной тройки. Таким образом, TransE *предпочитает* более низкие оценки для положительных ребер и большие оценки для отрицательных ребер.

Что касается параметра $\gamma$, он используется для обеспечения того, чтобы оценка положительного ребра отличалась от оценки отрицательного ребра как минимум на $\gamma$.

Итого алгоритм TransE выглядит следующим образом:

![](https://production-media.paperswithcode.com/methods/Screen_Shot_2020-05-27_at_12.01.23_AM.png)

Что касается реализации модели, можно инициализировать $\textbf l$ и $\textbf e$ в соответствии с приведенным выше псевдокодом. Чтобы вычислить $d(\textbf{h} + \textbf{l}, \textbf t)$, нужно взять L2-норму $\textbf h + \textbf l - \textbf t$.

*Примечание: для повышения производительности можно нормализовать $\textbf e$ каждую эпоху, а не каждый мини-батч.*

__Вспомогательные функции:__

Одним из ключевых аспектов обучения модели является создание измененных троек путем замены head или tail случайным объектом:

In [66]:
def create_neg_edge_index(edge_index, edge_type, num_entities):
    head_or_tail = torch.randint(high=2, size=edge_type.size(),
                                 device=device)
    rand_entities = torch.randint(high=num_entities,
                                  size=edge_type.size(), device=device)
    # change when 1, otherwise regular head
    heads = torch.where(head_or_tail == 1, rand_entities,
                        edge_index[0, :])
    # change when 0, otherwise regular tail
    tails = torch.where(head_or_tail == 0, rand_entities,
                        edge_index[1, :])
    return torch.stack([heads, tails], dim=0)

Оценивать качество будем по Hits@10, Mean Rank и MRR (mean reciprocal rank).

Hits@10 = $\frac{|\{r \in P | r \leq 10\}|}{|P|}$, где $|P|$ — количество оценок, а $r$ — ранг.

Mean Rank = $\frac{1}{|P|}\sum_{r \in P}r$

MMR = $\frac{1}{|P|}\sum_{r \in P}\frac{1}{r}$

Подробнее о метриках можно узнать [здесь](https://arxiv.org/pdf/2002.06914.pdf).

In [67]:
def mrr(predictions, gt):
    indices = predictions.argsort()
    return (1.0 / (indices == gt).nonzero()[:, 1].float().add(1.0)).sum().item()


def mr(predictions, gt):
    indices = predictions.argsort()
    return ((indices == gt).nonzero()[:, 1].float().add(1.0)).sum().item()


def hit_at_k(predictions, gt, device, k=10):
    zero_tensor = torch.tensor([0], device=device)
    one_tensor = torch.tensor([1], device=device)
    _, indices = predictions.topk(k=k, largest=False)
    return torch.where(indices == gt, one_tensor, zero_tensor).sum().item()

__Требуется__ добиться качества хотя бы 0.17 MRR и 0.30 Hits@10

In [68]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

from torch import Tensor
from typing import Union, Tuple, Optional
from torch_geometric.typing import (OptPairTensor, Adj, Size, NoneType,
                                    OptTensor)

from torch.nn import Parameter, Linear
from torch.utils.data import Dataset, DataLoader
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax

from tqdm import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [69]:
def load_inverse_dict(path):
    dict = {}

    for line in open(path, 'r').readlines()[1:]:
        key, value = line.strip().split('\t')
        dict[value] = int(key)

    return dict


class FB15K_Dataset(Dataset):
    def __init__(self, path, entity2id, relation2id):
        self.entity2id = entity2id
        self.relation2id = relation2id

        with open(path, "r") as f:
            self.data = [line[:-1].split("\t") for line in f]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item_id):
        head, relation, tail = self.data[item_id]
        head_id = self.entity2id[head] if head in self.entity2id else len(self.entity2id)
        relation_id = self.relation2id[relation] if relation in self.relation2id else len(self.relation2id)
        tail_id = self.entity2id[tail] if tail in self.entity2id else len(self.entity2id)
        return head_id, relation_id, tail_id

In [70]:
entity2id = load_inverse_dict('data/FB15k-237/raw/entities.dict')
relation2id = load_inverse_dict('data/FB15k-237/raw/relations.dict')

train_dataset = FB15K_Dataset('data/FB15k-237/raw/train.txt', entity2id, relation2id)
dev_dataset = FB15K_Dataset('data/FB15k-237/raw/valid.txt', entity2id, relation2id)
test_dataset = FB15K_Dataset('data/FB15k-237/raw/test.txt', entity2id, relation2id)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [71]:
class TransE(nn.Module):
    def __init__(self, n_entity, n_relation, norm=1, emb_size=100, margin=1.0, device=True):
        super(TransE, self).__init__()
        self.device = device

        self.n_entity = n_entity
        self.n_relation = n_relation

        self.norm = norm
        self.emb_size = emb_size
        
        # Эмбеддинги для сущностей и отношений
        self.entity_embedding = nn.Embedding(
            num_embeddings=self.n_entity+1,
            embedding_dim=self.emb_size,
            padding_idx=self.n_entity,
            _weight=torch.empty(
                self.n_entity+1, self.emb_size
            ).uniform_(-6 / np.sqrt(self.emb_size), 6 / np.sqrt(self.emb_size))
        )
        self.relation_embedding = nn.Embedding(
            num_embeddings=self.n_relation+1,
            embedding_dim=self.emb_size,
            padding_idx=self.n_relation,
            _weight=torch.empty(
                self.n_relation+1, self.emb_size
            ).uniform_(-6 / np.sqrt(self.emb_size), 6 / np.sqrt(self.emb_size))
        )
        
        # Нормализация весов отношений
        self.relation_embedding.weight.data[:-1, :].div_(self.relation_embedding.weight.data[:-1, :].norm(p=1, dim=1, keepdim=True))
         
        self.criterion = nn.MarginRankingLoss(margin=margin, reduction='none')

    def forward(self, positive_triple, negative_triple):
        # нормализация по сущностям
        self.entity_embedding.weight.data[:-1, :].div_(self.entity_embedding.weight.data[:-1, :].norm(p=2, dim=1, keepdim=True))
        
        # оценки для положительных и отрицательных троек
        positive_score = (
                self.entity_embedding(positive_triple[:, 0])
                + self.relation_embedding(positive_triple[:, 1])
                - self.entity_embedding(positive_triple[:, 2])
        ).norm(p=self.norm, dim=1)
        negative_score = (
                self.entity_embedding(negative_triple[:, 0])
                + self.relation_embedding(negative_triple[:, 1])
                - self.entity_embedding(negative_triple[:, 2])
        ).norm(p=self.norm, dim=1)

        y = torch.tensor([-1.], dtype=torch.float, device=self.device)

        return self.criterion(positive_score, negative_score, y)
    
    def predict(self, triplets):
        # предикт для троек
        return (
                self.entity_embedding(triplets[:, 0])
                + self.relation_embedding(triplets[:, 1])
                - self.entity_embedding(triplets[:, 2])
        ).norm(p=self.norm, dim=1)

In [72]:
weight_path="best_weights.pth"
EPOCH = 10
learning_rate = 0.001

# norm - l_k нормализация
# dim - размер эмбеддинга
# margin - отступ
model = TransE(n_entity=len(entity2id), n_relation=len(relation2id), emb_size=100, norm=2, margin=1, device=device)
model.to(device)
# С Adam обучалось быстрее всего
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [73]:
def eval(model, data_loader, device, chunk_size=1024):
    model.eval()
    count = 0
    hit_at_10, mrr_total = 0.0, 0.0
    
    # идентификаторы сущностей
    entity_ids = torch.arange(model.n_entity, device=device).unsqueeze(0)

    with torch.no_grad():
        for head, rel, tail in tqdm(data_loader):
            current_batch_size = head.size(0)
            head, rel, tail = head.to(device), rel.to(device), tail.to(device)
            b_size = head.size(0)
            entities = entity_ids.repeat(b_size, 1)

            heads = head.reshape(-1, 1).repeat(1, entities.size(1))
            relations = rel.reshape(-1, 1).repeat(1, entities.size(1))
            tails = tail.reshape(-1, 1).repeat(1, entities.size(1))

            tails_predictions_list = []
            heads_predictions_list = []
            
            # Обрабатываю триплеты по частям, чтобы ничего не сломалось
            # на валидации и тесте работает без этого, но при работе с тестом код падает
            def process_in_chunks(triplets, prediction_list):
                total_triplets = triplets.size(0)
                for i in range(0, total_triplets, chunk_size):
                    chunk = triplets[i:i + chunk_size]
                    preds = model.predict(chunk)
                    prediction_list.append(preds)
                return torch.cat(prediction_list, dim=0)
            
            # Предикты для tails и head
            triplets_tails = torch.stack((heads, relations, entities), dim=2).reshape(-1, 3)
            tails_predictions = process_in_chunks(triplets_tails, tails_predictions_list)
            tails_predictions = tails_predictions.reshape(current_batch_size, -1)

            triplets_heads = torch.stack((entities, relations, tails), dim=2).reshape(-1, 3)
            heads_predictions = process_in_chunks(triplets_heads, heads_predictions_list)
            heads_predictions = heads_predictions.reshape(current_batch_size, -1)

            predictions = torch.cat((tails_predictions, heads_predictions), dim=0)
            ground_truth_entity_id = torch.cat((tail.reshape(-1, 1), head.reshape(-1, 1)))
            hit_at_10 += hit_at_k(predictions, ground_truth_entity_id, device=device, k=10)
            mrr_total += mrr(predictions, ground_truth_entity_id)
            
            # Метрики
            hit_at_10 += hit_at_k(predictions, ground_truth_entity_id, device=device, k=10)
            mrr_total += mrr(predictions, ground_truth_entity_id)
            count += predictions.size(0)

    hit_at_10_score = (hit_at_10 / count)
    mrr_score = (mrr_total / count)
    return hit_at_10_score, mrr_score

In [74]:
best_hits = 0.
for epoch in range(1, EPOCH + 1):
    model.train()
    epoch_loss = 0
    for head, rel, tail in train_loader:
        head, rel, tail = head.to(device), rel.to(device), tail.to(device)

        positive_triples = torch.stack((head, rel, tail), dim=1)
        
        # Использую авторскую функцию
        broken_heads, broken_tails = create_neg_edge_index(torch.stack((head, tail), dim=0), rel, model.n_entity)
        negative_triples = torch.stack((broken_heads, rel, broken_tails), dim=1)
        
        optimizer.zero_grad()
        loss = model(positive_triples, negative_triples)[0].mean()
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()

    # Валидация
    # Работает долго, поэтому раз в n итераций
    print(f'Epoch {epoch}, train loss={epoch_loss}')
    if epoch % 2 == 0: # Метрики вычисляются слишком долго
        hit_at_10_score_val, mrr_score_val = eval(model, dev_loader, device)
        print(f'Val: hit10={hit_at_10_score_val}, mrr={mrr_score_val}')
        print()
        
        if hit_at_10_score_val > best_hits:
            best_hits = hit_at_10_score_val
            torch.save(model.state_dict(), weight_path)

Epoch 1, train loss=2106.670596599579
Epoch 2, train loss=2050.389019846916


100%|██████████| 137/137 [01:23<00:00,  1.64it/s]


Val: hit10=0.16053607071571144, mrr=0.0954439993424058
Epoch 3, train loss=1970.9582977890968
Epoch 4, train loss=1938.1018300056458


100%|██████████| 137/137 [01:19<00:00,  1.73it/s]


Val: hit10=0.2656401482748788, mrr=0.17224226734867318
Epoch 5, train loss=1876.2294586896896
Epoch 6, train loss=1853.329884648323


...

Val: hit10=0.338865126889079, mrr=0.21145811915771554
Epoch 7, train loss=1797.103194952011
Epoch 8, train loss=1755.0016440153122


100%|██████████| 137/137 [01:22<00:00,  1.67it/s]


Val: hit10=0.3731964642144283, mrr=0.225257779337043
Epoch 9, train loss=1686.4830737113953
Epoch 10, train loss=1664.4351626038551


100%|██████████| 137/137 [01:20<00:00,  1.71it/s]

Val: hit10=0.4048474479612204, mrr=0.23775006499017173


In [75]:
final_model = model.load_state_dict(torch.load(weight_path, weights_only=True))
hit_at_10_score_test, mrr_score_test = eval(model, test_loader, device)
print(f'Val: hit10={hit_at_10_score_test}, mrr={mrr_score_test}')

100%|██████████| 160/160 [01:27<00:00,  1.82it/s]

Val: hit10=0.39699990227694715, mrr=0.23411506769140597


Требуемые метрики получены на тесте

### 1.1 Вопрос о нормализации (2 балла)

Попробуйте обучить TransE без пятой строчки алгоритма (без нормализации по сущностям). Что происходит с обучением? Зачем требуется эта строка?

In [76]:
class TransENoNorm(nn.Module):
    def __init__(self, n_entity, n_relation, norm=1, emb_size=100, margin=1.0, device=True):
        super(TransENoNorm, self).__init__()
        self.device = device

        self.n_entity = n_entity
        self.n_relation = n_relation

        self.norm = norm
        self.emb_size = emb_size
        
        # Эмбеддинги для сущностей и отношений
        self.entity_embedding = nn.Embedding(
            num_embeddings=self.n_entity+1,
            embedding_dim=self.emb_size,
            padding_idx=self.n_entity,
            _weight=torch.empty(
                self.n_entity+1, self.emb_size
            ).uniform_(-6 / np.sqrt(self.emb_size), 6 / np.sqrt(self.emb_size))
        )
        self.relation_embedding = nn.Embedding(
            num_embeddings=self.n_relation+1,
            embedding_dim=self.emb_size,
            padding_idx=self.n_relation,
            _weight=torch.empty(
                self.n_relation+1, self.emb_size
            ).uniform_(-6 / np.sqrt(self.emb_size), 6 / np.sqrt(self.emb_size))
        )
        
        # Нормализация весов отношений
        self.relation_embedding.weight.data[:-1, :].div_(self.relation_embedding.weight.data[:-1, :].norm(p=1, dim=1, keepdim=True))
         
        self.criterion = nn.MarginRankingLoss(margin=margin, reduction='none')

    def forward(self, positive_triple, negative_triple):
        # Убрал нормализацию по сущностям
        #self.entity_embedding.weight.data[:-1, :].div_(self.entity_embedding.weight.data[:-1, :].norm(p=2, dim=1, keepdim=True))
        
        # оценки для положительных и отрицательных троек
        positive_score = (
                self.entity_embedding(positive_triple[:, 0])
                + self.relation_embedding(positive_triple[:, 1])
                - self.entity_embedding(positive_triple[:, 2])
        ).norm(p=self.norm, dim=1)
        negative_score = (
                self.entity_embedding(negative_triple[:, 0])
                + self.relation_embedding(negative_triple[:, 1])
                - self.entity_embedding(negative_triple[:, 2])
        ).norm(p=self.norm, dim=1)

        y = torch.tensor([-1.], dtype=torch.float, device=self.device)

        return self.criterion(positive_score, negative_score, y)
    
    def predict(self, triplets):
        # предикт для троек
        return (
                self.entity_embedding(triplets[:, 0])
                + self.relation_embedding(triplets[:, 1])
                - self.entity_embedding(triplets[:, 2])
        ).norm(p=self.norm, dim=1)

In [77]:
weight_path = "best_weights_no_norm.pth"
EPOCH = 10
learning_rate = 0.001

# norm - l_k нормализация
# dim - размер эмбеддинга
# margin - отступ
model = TransENoNorm(n_entity=len(entity2id), n_relation=len(relation2id), emb_size=100, norm=2, margin=1, device=device)
model.to(device)
# С Adam обучалось быстрее всего
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [78]:
best_hits = 0.
for epoch in range(1, EPOCH + 1):
    model.train()
    epoch_loss = 0
    for head, rel, tail in train_loader:
        head, rel, tail = head.to(device), rel.to(device), tail.to(device)

        positive_triples = torch.stack((head, rel, tail), dim=1)

        # Использую авторскую функцию
        broken_heads, broken_tails = create_neg_edge_index(torch.stack((head, tail), dim=0), rel, model.n_entity)
        negative_triples = torch.stack((broken_heads, rel, broken_tails), dim=1)

        optimizer.zero_grad()
        loss = model(positive_triples, negative_triples)[0].mean()

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    # Валидация
    # Работает долго, поэтому раз в n итераций
    print(f'Epoch {epoch}, train loss={epoch_loss}')
    if epoch % 2 == 0:  # Метрики вычисляются слишком долго
        hit_at_10_score_val, mrr_score_val = eval(model, dev_loader, device)
        print(f'Val: hit10={hit_at_10_score_val}, mrr={mrr_score_val}')
        print()

        if hit_at_10_score_val > best_hits:
            best_hits = hit_at_10_score_val
            torch.save(model.state_dict(), weight_path)

Epoch 1, train loss=2026.5538494586945
Epoch 2, train loss=1904.3653337955475


100%|██████████| 137/137 [01:17<00:00,  1.77it/s]


Val: hit10=0.08662674650698603, mrr=0.04020525038599254
Epoch 3, train loss=1827.6889173984528
Epoch 4, train loss=1785.2327275276184


100%|██████████| 137/137 [01:36<00:00,  1.41it/s]


Val: hit10=0.1508411747932706, mrr=0.06691357858982527
Epoch 5, train loss=1745.7671418190002
Epoch 6, train loss=1683.3792753219604


...

Val: hit10=0.17456515540347875, mrr=0.07541732877824325
Epoch 7, train loss=1645.341201543808
Epoch 8, train loss=1593.2606382369995


100%|██████████| 137/137 [01:29<00:00,  1.52it/s]


Val: hit10=0.1855717137154263, mrr=0.07892627947208784
Epoch 9, train loss=1585.6613001823425
Epoch 10, train loss=1567.5754010677338


100%|██████████| 137/137 [01:25<00:00,  1.60it/s]

Val: hit10=0.18887938408896493, mrr=0.0778330839628639


In [79]:
final_model = model.load_state_dict(torch.load(weight_path, weights_only=True))
hit_at_10_score_test, mrr_score_test = eval(model, test_loader, device)
print(f'Val: hit10={hit_at_10_score_test}, mrr={mrr_score_test}')

100%|██████████| 160/160 [01:36<00:00,  1.67it/s]

Val: hit10=0.18865435356200527, mrr=0.07601681061298002


*Ответ*: Я смотрел только первые 100 эпох, но уже на них видно, что модель обучается чуть хуже чем с ней.
 Если сравнивать ту же модель, но с нормализацией, лучшие результаты за 100 эпох:
100 эпох с нормализацией: hit10=0.39699990227694715, mrr=0.23411506769140597
100 эпох без нормализации: Val: hit10=0.18865435356200527, mrr=0.07601681061298002

Без нормализации значения эмбеддингов начинают неконтролируемо расти, что приводит к нестабильности обучения и, следовательно, качестве модели. Вместе с тем, модель начинает полагаться на масштаб векторов, а не на их направление, что снижает важность эмбеддингов. В результате модель не способна правильно оценивать сходства сущностей, что видно по плохим метрикам. К тому же модель обучается гораздо дольше (с точки зрения эпох)

## 2. Нейросеть на гетерогенных данных (3 баллов)

Возьмите один из 2 датасетов (Freebase/ синтетический датасет hetero_graph далее)

In [1]:
import numpy as np
import torch
import dgl

n_users = 1000
n_items = 500
n_follows = 3000
n_clicks = 5000
n_dislikes = 500
n_hetero_features = 10
n_user_classes = 5
n_max_clicks = 10

follow_src = np.random.randint(0, n_users, n_follows)
follow_dst = np.random.randint(0, n_users, n_follows)
click_src = np.random.randint(0, n_users, n_clicks)
click_dst = np.random.randint(0, n_items, n_clicks)
dislike_src = np.random.randint(0, n_users, n_dislikes)
dislike_dst = np.random.randint(0, n_items, n_dislikes)

hetero_graph = dgl.heterograph({
    ('user', 'follow', 'user'): (follow_src, follow_dst),
    ('user', 'followed-by', 'user'): (follow_dst, follow_src),
    ('user', 'click', 'item'): (click_src, click_dst),
    ('item', 'clicked-by', 'user'): (click_dst, click_src),
    ('user', 'dislike', 'item'): (dislike_src, dislike_dst),
    ('item', 'disliked-by', 'user'): (dislike_dst, dislike_src)})

hetero_graph.nodes['user'].data['feature'] = torch.randn(n_users, n_hetero_features)
hetero_graph.nodes['item'].data['feature'] = torch.randn(n_items, n_hetero_features)
hetero_graph.nodes['user'].data['label'] = torch.randint(0, n_user_classes, (n_users,))
hetero_graph.edges['click'].data['label'] = torch.randint(1, n_max_clicks, (n_clicks,)).float()
# randomly generate training masks on user nodes and click edges
hetero_graph.nodes['user'].data['train_mask'] = torch.zeros(n_users, dtype=torch.bool).bernoulli(0.6)
hetero_graph.edges['click'].data['train_mask'] = torch.zeros(n_clicks, dtype=torch.bool).bernoulli(0.6)

Используя любую библиотеку (torch geometry, dgl, stellargraph) соберите нейронную сеть и обучите ее (решать задачу Node Classification) на одном из двух датасетов выше.
Буду делать по документации https://docs.dgl.ai/guide/training.html

In [2]:
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
class RGCN(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, rel_names):
        super().__init__()

        self.conv1 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(in_feats, hid_feats)
            for rel in rel_names}, aggregate='sum')
        self.conv2 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(hid_feats, out_feats)
            for rel in rel_names}, aggregate='sum')

    def forward(self, graph, inputs):
        h = self.conv1(graph, inputs)
        h = {k: F.relu(v) for k, v in h.items()}
        h = self.conv2(graph, h)
        return h

In [3]:
model = RGCN(n_hetero_features, 20, n_user_classes, hetero_graph.etypes)
user_feats = hetero_graph.nodes['user'].data['feature']
item_feats = hetero_graph.nodes['item'].data['feature']
labels = hetero_graph.nodes['user'].data['label']
train_mask = hetero_graph.nodes['user'].data['train_mask']

In [4]:
node_features = {'user': user_feats, 'item': item_feats}
h_dict = model(hetero_graph, {'user': user_feats, 'item': item_feats})
h_user = h_dict['user']
h_item = h_dict['item']

In [5]:
opt = torch.optim.Adam(model.parameters())

train_epoch = 100

for epoch in range(train_epoch):
    model.train()

    logits = model(hetero_graph, node_features)['user']

    loss = F.cross_entropy(logits[train_mask], labels[train_mask])
    
    opt.zero_grad()
    loss.backward()
    opt.step()
    print(loss.item())

2.1442794799804688
2.1160073280334473
2.088643789291382
2.0622215270996094
2.0367558002471924
2.0122640132904053
1.9887603521347046
1.9662587642669678
1.9447709321975708
1.9242748022079468
1.90479576587677
1.8863213062286377
1.8688395023345947
1.8523207902908325
1.8367713689804077
1.8221595287322998
1.808450698852539
1.7956206798553467
1.783621907234192
1.7724181413650513
1.7619811296463013
1.7522600889205933
1.7432132959365845
1.7347960472106934
1.726955771446228
1.7196460962295532
1.7128269672393799
1.7064589262008667
1.7005069255828857
1.6949100494384766
1.689650058746338
1.6846897602081299
1.6799920797348022
1.6755244731903076
1.6712555885314941
1.6671662330627441
1.6632378101348877
1.6594499349594116
1.6557834148406982
1.6522296667099
1.648767113685608
1.6453922986984253
1.6420857906341553
1.6388286352157593
1.635622501373291
1.6324572563171387
1.6293264627456665
1.626230001449585
1.6231573820114136
1.6201180219650269
1.6170974969863892
1.6140981912612915
1.6111187934875488
1.6081

Вроде кроме обучения ничего не требовалось, так что оставлю этот, минимальный вариант

(1 балл) Обучите нейросеть решать задачу link prediction

In [6]:
# Также, по той же самой документации реализую link prediction
import dgl.function as fn

class HeteroDotProductPredictor(nn.Module):
    def forward(self, graph, h, etype):
        # h contains the node representations for each node type computed from
        # the GNN defined in the previous section (Section 5.1).
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(fn.u_dot_v('h', 'h', 'score'), etype=etype)
            return graph.edges[etype].data['score']

In [7]:
def construct_negative_graph(graph, k, etype):
    utype, _, vtype = etype
    src, dst = graph.edges(etype=etype)
    neg_src = src.repeat_interleave(k)
    neg_dst = torch.randint(0, graph.num_nodes(vtype), (len(src) * k,))
    return dgl.heterograph(
        {etype: (neg_src, neg_dst)},
        num_nodes_dict={ntype: graph.num_nodes(ntype) for ntype in graph.ntypes})

In [8]:
class Model(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, rel_names):
        super().__init__()
        self.sage = RGCN(in_features, hidden_features, out_features, rel_names)
        self.pred = HeteroDotProductPredictor()
    def forward(self, g, neg_g, x, etype):
        h = self.sage(g, x)
        return self.pred(g, h, etype), self.pred(neg_g, h, etype)

In [9]:
def compute_loss(pos_score, neg_score):
    # Margin loss
    n_edges = pos_score.shape[0]
    return (1 - pos_score + neg_score.view(n_edges, -1)).clamp(min=0).mean()

In [10]:
k = 5
model = Model(10, 20, 5, hetero_graph.etypes)
user_feats = hetero_graph.nodes['user'].data['feature']
item_feats = hetero_graph.nodes['item'].data['feature']
node_features = {'user': user_feats, 'item': item_feats}
opt = torch.optim.Adam(model.parameters())

In [11]:

for epoch in range(500):
    negative_graph = construct_negative_graph(hetero_graph, k, ('user', 'click', 'item'))
    pos_score, neg_score = model(hetero_graph, negative_graph, node_features, ('user', 'click', 'item'))
    loss = compute_loss(pos_score, neg_score)
    opt.zero_grad()
    loss.backward()
    opt.step()
    print(loss.item())


1.3535784482955933
1.3282545804977417
1.2992197275161743
1.2697863578796387
1.256778359413147
1.2323118448257446
1.2187252044677734
1.210707426071167
1.1845128536224365
1.1654188632965088
1.1582081317901611
1.1482540369033813
1.1384446620941162
1.1278752088546753
1.1179730892181396
1.1115925312042236
1.1072744131088257
1.101805567741394
1.092278242111206
1.0815913677215576
1.0779883861541748
1.0772298574447632
1.0710248947143555
1.0695019960403442
1.0663349628448486
1.0541067123413086
1.0541603565216064
1.0499916076660156
1.045789361000061
1.039341688156128
1.0431368350982666
1.036928415298462
1.0314584970474243
1.029486894607544
1.02236008644104
1.024398684501648
1.0171151161193848
1.0200661420822144
1.0107725858688354
1.0092138051986694
1.0055047273635864
1.0031468868255615
1.003190279006958
1.0014487504959106
0.9993492364883423
0.9976548552513123
0.9973813891410828
0.9906138181686401
0.9962586760520935
0.9939156770706177
0.9891543984413147
0.9865503311157227
0.9849533438682556
0.986

Опять же, ничего, кроме обучения, не требовалось


(1 балл) возьмите еще какой-нибудь гетерогенный датасет (не маленький и не синтетический) и обучите на нем


In [12]:
import os
import requests
import zipfile

url = "https://files.grouplens.org/datasets/movielens/ml-25m.zip"
output_dir = "movielens_25m"
zip_file = "ml-25m.zip"

if not os.path.exists(zip_file):
    print(f"Скачивание {zip_file}...")
    response = requests.get(url, stream=True)
    with open(zip_file, "wb") as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
    print("Скачивание завершено.")

In [13]:
if not os.path.exists(output_dir):
    print(f"Распаковка {zip_file}...")
    with zipfile.ZipFile(zip_file, "r") as zip_ref:
        zip_ref.extractall(output_dir)
    print(f"Файлы распакованы в папку {output_dir}.")
else:
    print(f"Папка {output_dir} уже существует.")

Распаковка ml-25m.zip...
Файлы распакованы в папку movielens_25m.


In [18]:
import pandas as pd

ratings = pd.read_csv('movielens_25m/ml-25m/ratings.csv')
movies = pd.read_csv('movielens_25m/ml-25m/movies.csv')

n_users = ratings['userId'].nunique()
n_items = ratings['movieId'].nunique()

user_to_idx = {user_id: idx for idx, user_id in enumerate(ratings['userId'].unique())}
item_to_idx = {movie_id: idx for idx, movie_id in enumerate(ratings['movieId'].unique())}

ratings['user_idx'] = ratings['userId'].map(user_to_idx)
ratings['item_idx'] = ratings['movieId'].map(item_to_idx)

click_src = ratings['user_idx'].values
click_dst = ratings['item_idx'].values

graph_data = {
    ('user', 'click', 'item'): (click_src, click_dst),
    ('item', 'clicked-by', 'user'): (click_dst, click_src),
}
hetero_graph = dgl.heterograph(graph_data)

n_hetero_features = 10  # Количество признаков
hetero_graph.nodes['user'].data['feature'] = torch.randn(n_users, n_hetero_features)
hetero_graph.nodes['item'].data['feature'] = torch.randn(n_items, n_hetero_features)
hetero_graph.nodes['user'].data['label'] = torch.randint(0, 5, (n_users,))
hetero_graph.edges['click'].data['label'] = torch.tensor(ratings['rating'].values, dtype=torch.float32)
hetero_graph.nodes['user'].data['train_mask'] = torch.zeros(n_users, dtype=torch.bool).bernoulli(0.6)
hetero_graph.edges['click'].data['train_mask'] = torch.zeros(len(click_src), dtype=torch.bool).bernoulli(0.6)

print(hetero_graph)

Graph(num_nodes={'item': 59047, 'user': 162541},
      num_edges={('item', 'clicked-by', 'user'): 25000095, ('user', 'click', 'item'): 25000095},
      metagraph=[('item', 'user', 'clicked-by'), ('user', 'item', 'click')])


In [20]:
model = RGCN(n_hetero_features, 20, n_user_classes, hetero_graph.etypes)
user_feats = hetero_graph.nodes['user'].data['feature']
item_feats = hetero_graph.nodes['item'].data['feature']
labels = hetero_graph.nodes['user'].data['label']
train_mask = hetero_graph.nodes['user'].data['train_mask']
node_features = {'user': user_feats, 'item': item_feats}
h_dict = model(hetero_graph, {'user': user_feats, 'item': item_feats})
h_user = h_dict['user']
h_item = h_dict['item']
opt = torch.optim.Adam(model.parameters())

train_epoch = 5000

for epoch in range(train_epoch):
    model.train()

    logits = model(hetero_graph, node_features)['user']

    loss = F.cross_entropy(logits[train_mask], labels[train_mask])

    opt.zero_grad()
    loss.backward()
    opt.step()
    print(loss.item())

1.6094943284988403
1.6094820499420166
1.6094714403152466
1.6094627380371094
1.6094557046890259
1.609450101852417
1.6094454526901245
1.6094417572021484
1.6094386577606201
1.609436273574829
1.609433889389038
1.6094318628311157
1.6094300746917725
1.6094284057617188
1.6094269752502441
1.6094257831573486
1.6094247102737427
1.6094235181808472
1.6094225645065308
1.6094216108322144
1.6094204187393188
1.609419345855713
1.6094180345535278
1.6094167232513428
1.6094157695770264
1.6094145774841309
1.6094133853912354
1.6094123125076294
1.6094111204147339
1.6094101667404175
1.609409213066101
1.6094084978103638
1.6094075441360474
1.6094069480895996
1.6094063520431519
1.609405755996704
1.6094050407409668
1.6094043254852295
1.6094039678573608
1.609403133392334
1.6094025373458862
1.6094019412994385
1.6094011068344116
1.6094003915786743
1.609399676322937
1.6093991994857788
1.6093982458114624
1.609397530555725
1.6093969345092773
1.6093965768814087
1.6093956232070923
1.609395146369934
1.6093945503234863
1.6

Из-за большего объёма данных обучается медленно.
Опять же, требовалось только обучить)

**БОНУСЫ:**
(2 балла) реализуйте самостоятельно Relational GCN и продемонстрируйте работоспобность вашего слоя.
Это я уже не успел